In [ ]:
#
# Notebook that generates summary information for tracks in a particular media file.
#
# The following outputs will be generated:
# - Summary .csv file providing track information in the media
# - For each track, a folder containing images of each detection, a track animated graphic
#   and a summary .csv file providing detection information
#
# Review the "General Notebook Settings" cell for parameters required to be edited by the user
# that pertain with most of the cells in this notebook.
#
# Once the track data has been collected, review the "Track Detail" cell for parameters to be
# edited by the user to output specific track data (e.g. generate images for the track)
#

In [ ]:
#
# Install and import relevant modules
#
import os
import shutil
import sys
from datetime import datetime
from enum import Enum
from types import SimpleNamespace

!{sys.executable} -m pip install -I tator>=0.0.21
!{sys.executable} -m pip install progressbar2
import tator
import progressbar

!{sys.executable} -m pip install pandas
import pandas as pd

In [ ]:
print(tator.__version__) # Want to make sure we've installed the correct version of tator

In [ ]:
# ****************************************************************
# EDITABLE SECTION: General Notebook Settings
# ****************************************************************

# Tator URL
host = "https://.tator.io"

# User-specific access token
token = ""

# Media id with tracks to process
media_id = 0

# Output directory that will contain all the track data
output_directory = './'

# Set to true to generate images using the following endpoints/utilities:
#  localization_graphic_endpoint
#  state_graphic endpoint
#  full_state_graphic utility function
generate_images = True

In [ ]:
#
# Grab project information
#

# Connect to tator
tator_api = tator.get_api(host=host, token=token)

# Get related media information
media = tator_api.get_media(id=media_id)

# Get the project ID
project_id = media.project

# Grab the localization type that is a box. It's assumed that this project
# has been set up to only have one localization box type (that will be the detections)
box_type_counts = 0
detection_type_id = None
localization_types = tator_api.get_localization_type_list(project=project_id)
for loc_type in localization_types:
    if loc_type.dtype == 'box':
        detection_type_id = loc_type.id
        box_type_counts += 1 
        
if box_type_counts > 1:
    print("WARNING: More than one 'box' type in localization type list")
    
if detection_type_id is None:
    print("ERROR: Could not find a 'box' type in localization type list")
    
# Grab the versions associated with this project
# This will be used later when collecting the detection data
versions = tator_api.get_version_list(project=project_id)

version_id_name_map = {}
for version in versions:
    version_id_name_map[version.id] = version.name

In [ ]:
#
# Get the tracks that are associated with the media
#
tracks = tator_api.get_state_list(project=project_id, media_id=[media_id])

In [ ]:
#
# Create a summary file that has high level information about each track
#
tracks = tator_api.get_state_list(project=project_id, media_id=[media_id])
track_data_list = []
for track in tracks:

    track_data = {}
    track_data['project_id'] = project_id
    track_data['media_id'] = media_id
    track_data['media_name'] = media.name
    track_data['track_id'] = track.id
    track_data['start_frame'] = track.frame
    track_data['number_of_detections'] = len(track.localizations)
    track_data['data_folder'] = os.path.join(output_directory,'tracks',f'track_{track.id}')
    track_data_list.append(track_data)
    print(track_data)
    
# Convert list of dictionaries into a pandas DataFrame and the save it as a csv in the output directory
# This .csv file will contain an entry for each track in this media
track_summary_df = pd.DataFrame(track_data_list)
os.makedirs(output_directory, exist_ok=True)
final_filename = os.path.join(output_directory, f'project_{project_id}_media_{media_id}_tracks_summary.csv')
track_summary_df.to_csv(final_filename, index=False)

In [ ]:
# ****************************************************************
# EDITABLE SECTION: Track Details
# ****************************************************************

# Set this to 'all' if all the tracks should be processed
# (i.e. track report created, images created if requested to do so)
# Otherwise, create a list of track ids (e.g. [1], [100,105,103]) to process
selected_track_ids = []

In [ ]:
#
# A new folder will be created in the output directory specifically for the track.
# All the detection information will be stored in a .csv file
# Images of the detections and an animated state graphic will be generated if requested
# These images will be stored in the track output folder.
#
if selected_track_ids == 'all':
    selected_tracks = tracks
else:
    selected_tracks = []
    for selected_id in selected_track_ids:
        for track in tracks:
            if track.id == selected_id:
                selected_tracks.append(track)
                break

for track in selected_tracks:
    
    # Create the directory that we will dump the data to
    track_output_directory = os.path.join(output_directory, f'track_{track.id}')
    os.makedirs(track_output_directory, exist_ok=True)

    # Loop through each detection in the track, grab information to be stored
    # in the .csv summary file, and create the thumbnail
    detection_data_list = []
    for detection_id in track.localizations:
            
        # Get the current track detection
        detection = tator_api.get_localization(id=detection_id)
            
        # Create the detection image using the localization graphic endpoint
        image_filename = 'N/A'
        if generate_images:
            image_path = tator_api.get_localization_graphic(
                id=detection.id,
                use_default_margins=False,
                margin_x=0,
                margin_y=0)
            image_filename = f'frame_{detection.frame:05d}_track_{track.id}_det_{detection.id}.png'
            image_folder = os.path.join(track_output_directory, 'localization_graphic_endpoint_images')
            target_path = os.path.join(image_folder, image_filename)
            os.makedirs(image_folder, exist_ok=True)
            shutil.move(image_path, target_path)

        # Get the mathcing version name
        version_name = ''
        if detection.version in version_id_name_map:
            version_name = version_id_name_map[detection.version]
        
        # Gather the data for the track summary file
        det_data = {}
        det_data['annotation_id'] = detection.id
        det_data['media_id'] = media.id
        det_data['media_name'] = media.name
        det_data['media_height'] = media.height
        det_data['media_width'] = media.width
        det_data['frame'] = detection.frame
        det_data['x'] = detection.x
        det_data['y'] = detection.y
        det_data['width'] = detection.width
        det_data['height'] = detection.height
        det_data['image'] = image_filename
        det_data['version'] = version_name
        det_data.update(detection.attributes)
        detection_data_list.append(det_data)
    
    # Create the animated state graphic associated with the track using the first set of localizations
    # Note, there is a cap of 100 frames
    if generate_images:
        
        image_path = tator_api.get_state_graphic(id=track.id, mode='animate', force_scale="240x240")
        image_filename = f'track_{track.id}_initial_timeline_animation.gif'
        image_folder = os.path.join(track_output_directory)
        target_path = os.path.join(image_folder, image_filename)
        os.makedirs(image_folder, exist_ok=True)
        shutil.move(image_path, target_path)
    
    # Generate of the a set of PIL.Image.Image objects using the full state graphic utility function
    if generate_images:
        pil_image_list = tator.util.full_state_graphic(api=tator_api, state_id=track.id, force_scale='224x224')

        # Save the image objects as pngs
        for index, image in enumerate(pil_image_list):
            image_filename = f'track_{track.id}_index_{index}.png'
            image_folder = os.path.join(track_output_directory, 'full_state_graphic_utility')
            os.makedirs(image_folder, exist_ok=True)
            image_filename = os.path.join(image_folder, image_filename)
            image.save(image_filename,"PNG")
    
    # Save the detection information to a track summary csv file
    track_summary_df = pd.DataFrame(detection_data_list)
    final_filename = os.path.join(track_output_directory, f'project_{project_id}_media_{media_id}_track_{track.id}_summary.csv')
    track_summary_df.to_csv(final_filename, index=False)
    